In [ ]:
pl_data = []
laliga_data = []
seria_data = []
bundes_data = []
ligue1_data = []

import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
headers = {
    'User-Agent': ''}

def get_team_results(team, team_url):
    response = requests.get(team_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    matches = soup.find_all('tr', class_='Table__TR Table__TR--sm Table__even')
    match_results = []
    for match in matches:
        date = match.find('div', class_='matchTeams').text if match.find('div', class_='matchTeams') else ''
        opponents=[]
        count = 0
        for opponent in match:
            opponent = match.find_all('a', class_='AnchorLink Table__Team')
            count = 0
            home = True
            for link in opponent:
                if team not in link.text:
                    opponents.append(link.text)
                    break
                count += 1
        if len(opponents) == 0:
            op = team
        else:
            op = opponents[0]
        if count == 0:
            home = False

        scores = []
        for result in match:
            result = match.find_all('a', class_='AnchorLink', href=True)
            for link in result:
                if ' - ' in link.text:
                    scores.append(link.text)
        res = scores[0]
        win = eval(res)
        r = '.'
        if home:
            if win > 0:
                r = 'Win'
            elif win == 0:
                r = 'Draw'
            else:
                r = 'Lose'
        else:
            if win < 0:
                r = 'Win'
            elif win == 0:
                r = 'Draw'
            else:
                r = 'Lose'

        competitions = []
        for comp in match:
            comp = match.find_all('td', class_='Table__TD')
            for link in comp:
                if 'English Premier League' in link.text:
                    competitions.append(link.text)
                elif 'Spanish LALIGA' in link.text:
                    competitions.append(link.text)
                elif 'Italian Serie A' in link.text:
                    competitions.append(link.text)
                elif 'German Bundesliga' in link.text:
                    competitions.append(link.text)
                elif 'French Ligue 1' in link.text:
                    competitions.append(link.text)

        if len(competitions) == 0:
            competitions.append('.')

        competition = competitions[0]

        h = 'Home'
        if not home:
            h = 'Away'

                # 홈 경기의 경우 eval(res)를 그대로 사용
        if home:
            if r == 'Win':
                result_ = eval(res)
            elif r == 'Draw':
                result_ = 0
            else:
                result_ =  eval(res)
        # 원정 경기의 경우 eval(res)에 -1을 곱해서 사용
        else:
            if r == 'Win':
                result_ = -eval(res)
            elif r == 'Draw':
                result_ = 0
            else:
                result_ = -eval(res)

        if competition == 'English Premier League':
            pl_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'Spanish LALIGA':
            laliga_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'Italian Serie A':
            seria_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'German Bundesliga':
            bundes_data.append({'Team': team, 'Opponent': op, 'Result': result_})
        elif competition == 'French Ligue 1':
            ligue1_data.append({'Team': team, 'Opponent': op, 'Result': result_})

        match_results.append({
            'Date': date,
            'Opponent': op,
            'Score': res,
            'Competition': competition,
            'Result': r,
            'Home': h
        })

    return match_results

team_urls = {
    # EPL
    'Arsenal': 'https://www.espn.com/soccer/team/results/_/id/359/season/2021',
    'Aston Villa': 'https://www.espn.com/soccer/team/results/_/id/362/season/2021',
    'Brentford': 'https://www.espn.com/soccer/team/results/_/id/337/season/2021',
    'Brighton & Hove Albion': 'https://www.espn.com/soccer/team/results/_/id/331/season/2021',
    'Burnley': 'https://www.espn.com/soccer/team/results/_/id/379/season/2021',
    'Chelsea': 'https://www.espn.com/soccer/team/results/_/id/363/season/2021',
    'Crystal Palace': 'https://www.espn.com/soccer/team/results/_/id/384/season/2021',
    'Everton': 'https://www.espn.com/soccer/team/results/_/id/368/season/2021',
    'Leeds United': 'https://www.espn.com/soccer/team/results/_/id/357/season/2021',
    'Leicester City': 'https://www.espn.com/soccer/team/results/_/id/375/season/2021',
    'Liverpool': 'https://www.espn.com/soccer/team/results/_/id/364/season/2021',
    'Manchester City': 'https://www.espn.com/soccer/team/results/_/id/382/season/2021',
    'Manchester United': 'https://www.espn.com/soccer/team/results/_/id/360/season/2021',
    'Newcastle United': 'https://www.espn.com/soccer/team/results/_/id/361/season/2021',
    'Norwich City': 'https://www.espn.com/soccer/team/results/_/id/381/season/2021',
    'Southampton': 'https://www.espn.com/soccer/team/results/_/id/376/season/2021',
    'Tottenham Hotspur': 'https://www.espn.com/soccer/team/results/_/id/367/season/2021',
    'Watford': 'https://www.espn.com/soccer/team/results/_/id/395/season/2021',
    'West Ham United': 'https://www.espn.com/soccer/team/results/_/id/371/season/2021',
    'Wolverhampton Wanderers': 'https://www.espn.com/soccer/team/results/_/id/380/season/2021',

    # LALIGA
    'Alavés': 'https://www.espn.com/soccer/team/results/_/id/96/season/2021',
    'Athletic Club': 'https://www.espn.com/soccer/team/results/_/id/93/season/2021',
    'Atlético Madrid': 'https://www.espn.com/soccer/team/results/_/id/1068/season/2021',
    'Barcelona': 'https://www.espn.com/soccer/team/results/_/id/83/season/2021',
    'Cádiz': 'https://www.espn.com/soccer/team/results/_/id/3842/season/2021',
    'Celta Vigo': 'https://www.espn.com/soccer/team/results/_/id/85/season/2021',
    'Elche': 'https://www.espn.com/soccer/team/results/_/id/3751/season/2021',
    'Espanyol': 'https://www.espn.com/soccer/team/results/_/id/88/season/2021',
    'Getafe': 'https://www.espn.com/soccer/team/results/_/id/2922/season/2021',
    'Granada': 'https://www.espn.com/soccer/team/results/_/id/3747/season/2021',
    'Levante': 'https://www.espn.com/soccer/team/results/_/id/1538/season/2021',
    'Mallorca': 'https://www.espn.com/soccer/team/results/_/id/84/season/2021',
    'Osasuna': 'https://www.espn.com/soccer/team/results/_/id/97/season/2021',
    'Rayo Vallecano': 'https://www.espn.com/soccer/team/results/_/id/101/season/2021',
    'Real Betis': 'https://www.espn.com/soccer/team/results/_/id/244/season/2021',
    'Real Madrid': 'https://www.espn.com/soccer/team/results/_/id/86/season/2021',
    'Real Sociedad': 'https://www.espn.com/soccer/team/results/_/id/89/season/2021',
    'Sevilla': 'https://www.espn.com/soccer/team/results/_/id/243/season/2021',
    'Valencia': 'https://www.espn.com/soccer/team/results/_/id/94/season/2021',
    'Villarreal': 'https://www.espn.com/soccer/team/results/_/id/102/season/2021',

    # SERIA A
    'AC Milan': 'https://www.espn.com/soccer/team/results/_/id/103/season/2021',
    'AS Roma': 'https://www.espn.com/soccer/team/results/_/id/104/season/2021',
    'Atalanta': 'https://www.espn.com/soccer/team/results/_/id/105/season/2021',
    'Bologna': 'https://www.espn.com/soccer/team/results/_/id/107/season/2021',
    'Cagliari': 'https://www.espn.com/soccer/team/results/_/id/2925/season/2021',
    'Empoli': 'https://www.espn.com/soccer/team/results/_/id/2574/season/2021',
    'Fiorentina': 'https://www.espn.com/soccer/team/results/_/id/109/season/2021',
    'Genoa': 'https://www.espn.com/soccer/team/results/_/id/3263/season/2021',
    'Hellas Verona': 'https://www.espn.com/soccer/team/results/_/id/119/season/2021',
    'Internazionale': 'https://www.espn.com/soccer/team/results/_/id/110/season/2021',
    'Juventus': 'https://www.espn.com/soccer/team/results/_/id/111/season/2021',
    'Lazio': 'https://www.espn.com/soccer/team/results/_/id/112/season/2021',
    'Napoli': 'https://www.espn.com/soccer/team/results/_/id/114/season/2021',
    'Salernitana': 'https://www.espn.com/soccer/team/results/_/id/3240/season/2021',
    'Sampdoria': 'https://www.espn.com/soccer/team/results/_/id/2734/season/2021',
    'Sassuolo': 'https://www.espn.com/soccer/team/results/_/id/3997/season/2021',
    'Spezia': 'https://www.espn.com/soccer/team/results/_/id/3692/season/2021',
    'Torino': 'https://www.espn.com/soccer/team/results/_/id/239/season/2021',
    'Udinese': 'https://www.espn.com/soccer/team/results/_/id/118/season/2021',
    'Venezia': 'https://www.espn.com/soccer/team/results/_/id/17530/season/2021',

    # BUNDESLIGEA
    'FC Augsburg': 'https://www.espn.com/soccer/team/results/_/id/3841/season/2021',
    'Hertha Berlin': 'https://www.espn.com/soccer/team/results/_/id/129/season/2021',
    'Union Berlin': 'https://www.espn.com/soccer/team/results/_/id/598/season/2021',
    'Werder Bremen': 'https://www.espn.com/soccer/team/results/_/id/137/season/2021',
    'Borussia Dortmund': 'https://www.espn.com/soccer/team/results/_/id/124/season/2021',
    'Eintracht Frankfurt': 'https://www.espn.com/soccer/team/results/_/id/125/season/2021',
    'SC Freiburg': 'https://www.espn.com/soccer/team/results/_/id/126/season/2021',
    'TSG Hoffenheim': 'https://www.espn.com/soccer/team/results/_/id/7911/season/2021',
    'FC Cologne': 'https://www.espn.com/soccer/team/results/_/id/122/season/2021',
    'RB Leipzig': 'https://www.espn.com/soccer/team/results/_/id/11420/season/2021',
    'Bayer Leverkusen': 'https://www.espn.com/soccer/team/results/_/id/131/season/2021',
    'Mainz': 'https://www.espn.com/soccer/team/results/_/id/2950/season/2021',
    'Borussia Monchengladbach': 'https://www.espn.com/soccer/team/results/_/id/268/season/2021',
    'Bayern Munich': 'https://www.espn.com/soccer/team/results/_/id/132/season/2021',
    'VfL Bochum': 'https://www.espn.com/soccer/team/results/_/id/121/season/2021',
    'VfB Stuttgart': 'https://www.espn.com/soccer/team/results/_/id/134/season/2021',
    'VfL Wolfsburg': 'https://www.espn.com/soccer/team/results/_/id/138/season/2021',
    'SpVgg Greuther Furth': 'https://www.espn.com/soccer/team/results/_/id/3070/season/2021',

    # LIGUE 1
    'Angers': 'https://www.espn.com/soccer/team/results/_/id/7868/season/2021',
    'Bordeaux': 'https://www.espn.com/soccer/team/results/_/id/159/season/2021',
    'Brest': 'https://www.espn.com/soccer/team/results/_/id/6997/season/2021',
    'Clermont Foot': 'https://www.espn.com/soccer/team/results/_/id/3171/season/2021',
    'Lens': 'https://www.espn.com/soccer/team/results/_/id/175/season/2021',
    'Lille': 'https://www.espn.com/soccer/team/results/_/id/166/season/2021',
    'Lorient': 'https://www.espn.com/soccer/team/results/_/id/273/season/2021',
    'Lyon': 'https://www.espn.com/soccer/team/results/_/id/167/season/2021',
    'Marseille': 'https://www.espn.com/soccer/team/results/_/id/176/season/2021',
    'Metz': 'https://www.espn.com/soccer/team/results/_/id/177/season/2021',
    'Monaco': 'https://www.espn.com/soccer/team/results/_/id/174/season/2021',
    'Montpellier': 'https://www.espn.com/soccer/team/results/_/id/274/season/2021',
    'Nantes': 'https://www.espn.com/soccer/team/results/_/id/165/season/2021',
    'Nice': 'https://www.espn.com/soccer/team/results/_/id/2502/season/2021',
    'Paris Saint-Germain': 'https://www.espn.com/soccer/team/results/_/id/160/season/2021',
    'Reims': 'https://www.espn.com/soccer/team/results/_/id/3243/season/2021',
    'Stade Rennais': 'https://www.espn.com/soccer/team/results/_/id/169/season/2021',
    'Saint-Etienne': 'https://www.espn.com/soccer/team/results/_/id/178/season/2021',
    'Strasbourg': 'https://www.espn.com/soccer/team/results/_/id/180/season/2021',
    'Troyes': 'https://www.espn.com/soccer/team/results/_/id/170/season/2021'
}

all_team_results = {}

for team, url in team_urls.items():
    results = get_team_results(team, url)
    all_team_results[team] = results

for team, results in all_team_results.items():
    df = pd.DataFrame(results)
    #print(f"\n{team} Match Results:")
    #print(df)

In [ ]:
for team, results in all_team_results.items():
    df = pd.DataFrame(results)
    df = df[df['Competition'] != '.']
    file_name = f'{team}_2122.csv'
    df.to_csv(file_name, index=False, encoding = "utf-8-sig")

In [ ]:
pl_df = pd.DataFrame(pl_data)
pl_df.to_csv('pl_df_2122.csv', index=False, encoding="utf-8-sig")

laliga_df = pd.DataFrame(laliga_data)
laliga_df.to_csv('laliga_df_2122.csv', index=False, encoding="utf-8-sig")

seria_df = pd.DataFrame(seria_data)
seria_df.to_csv('seria_df_2122.csv', index=False, encoding="utf-8-sig")

bundes_df = pd.DataFrame(bundes_data)
bundes_df.to_csv('bundes_df_2122.csv', index=False, encoding="utf-8-sig")

ligue1_df = pd.DataFrame(ligue1_data)
ligue1_df.to_csv('ligue1_df_2122.csv', index=False, encoding="utf-8-sig")
